<a href="https://colab.research.google.com/github/Napawan2005/NLP---Text-Preprocessing/blob/Test/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [ ]:
import pandas as pd
import nltk


In [ ]:
df = pd.read_csv('/content/spam.csv', encoding='latin1')

### `encoding='latin1'` คืออะไร?

เมื่อคอมพิวเตอร์เก็บข้อมูลที่เป็นตัวอักษร จะมีการเข้ารหัส (encode) ตัวอักษรเหล่านั้นให้อยู่ในรูปแบบของตัวเลข (ไบต์) และเมื่อต้องการอ่านข้อมูลกลับคืนมา ก็จะต้องมีการถอดรหัส (decode) ตัวเลขเหล่านั้นให้กลับมาเป็นตัวอักษรเดิม

- **`encoding`**: เป็นพารามิเตอร์ในฟังก์ชัน `pd.read_csv()` (และฟังก์ชันอื่นๆ ที่เกี่ยวข้องกับการอ่านไฟล์) ที่ใช้เพื่อบอกโปรแกรมว่าไฟล์ที่คุณกำลังอ่านนั้นถูกเข้ารหัสด้วยชุดอักขระแบบไหน
- **`latin1` (หรือ ISO-8859-1)**: เป็นมาตรฐานการเข้ารหัสชุดอักขระแบบหนึ่งที่นิยมใช้กันในอดีต โดยเฉพาะกับภาษาในกลุ่มยุโรปตะวันตก มันสามารถรองรับอักขระพิเศษบางตัวได้ดีกว่าการเข้ารหัสแบบ `ascii` แต่ไม่ครอบคลุมอักขระทั่วโลกเหมือน `utf-8`

#### ทำไมถึงต้องใช้ `encoding='latin1'`?

ข้อผิดพลาดที่คุณพบก่อนหน้านี้ (`UnicodeDecodeError: 'utf-8' codec can't decode bytes...`) เกิดขึ้นเพราะว่า `pandas` พยายามอ่านไฟล์ CSV ของคุณโดยใช้การเข้ารหัสแบบ **`utf-8`** ซึ่งเป็นค่าเริ่มต้นและเป็นชุดอักขระที่ครอบคลุมมากที่สุดในปัจจุบัน

แต่ถ้าไฟล์ CSV ของคุณถูกสร้างขึ้นหรือบันทึกด้วยการเข้ารหัสแบบอื่น (เช่น `latin1`) เมื่อ `pandas` พยายามถอดรหัสด้วย `utf-8` ก็จะเจอไบต์ที่ไม่สามารถตีความได้ ทำให้เกิดข้อผิดพลาดขึ้น

การระบุ `encoding='latin1'` เป็นการบอก `pandas` ให้ใช้ชุดอักขระ `latin1` ในการถอดรหัสไฟล์ ทำให้สามารถอ่านไฟล์ที่มีการเข้ารหัสแบบนั้นได้อย่างถูกต้องครับ

In [ ]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


---

วิเคราะห์ของ spam ว่าข้อมูลส่วนใหญ่มีลักษณะยังไง

In [24]:
df[df['v1'] == 'spam']

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN,NaN
...,...,...,...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,NaN,NaN,NaN
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,NaN,NaN,NaN
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,NaN,NaN,NaN
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,NaN,NaN,NaN


In [ ]:
df[df['v1'] == 'spam'].shape

(747, 5)

---
วิเคราะห์ข้อมูลเพื่อได้ข้อเปรียบเทียบที่ชัดเจนว่า โซนไหนเด่นเรื่องอะไร เพื่อทำความสะอาดข้อมูล

In [25]:
df[df['v1'] == 'ham']

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
...,...,...,...,...,...
5565,ham,Huh y lei...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
df[df['v1'] == 'ham'].shape

(4825, 5)

---

# ข้อสังเกตุที่ค้นพบเกี่ยวกับ Dataset นี้
- ถ้ามีเบอร์โทรให้เปลี่ยน เช่น 08452810075 เปลี่ยนเป็น [call]
- ถ้ามีลิงค์เช่น www.dbuk.net(row 12) หรือhttp://wap.xxxmobilemovieclub.com?n=QJKGIGHJJGCBL( row 15) ให้จำจัดทิ้ง/เปลี่ยนเป็น[URL]
- ถ้ามีตัวอักษรพิเศษและตามด้วยตัวเลขที่ไม่รู้จักให้ทำการจำกัดทิ้ง เช่น å£100,000 (row 12) ทำการจำกัดทิ้งหรือแทนที่ [Grand Total]
- การมีรหัสย่อแปลกๆเช่น 69888 (row 134) , 87121 (row 133) ทำการจำกัดทิ้งหรือแทนที่ [code]
- มีคำย่อ เช่น lol (ทำ normalization ไหม  ?)
- มีลัญลักษณ์ที่ใช้ html ปนอยู่ เช่น <#>(row 44) ทำการลบทิ้ง
- เปลี่ยนเป็นตังอักษระเป็นพิมพ์อย่างเดียวทั้งหมด